<a href="https://colab.research.google.com/github/LoniQin/lifelong-ml/blob/main/Running_Alpaca_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running Alpaca on Google Colab

## Install Dependencies

In [ ]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

## Import Libraries

In [2]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

## Loading model

In [3]:
tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

## Usage

In [27]:
def chat(input_text):
    text =f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    {input_text}

    ### Response:
    '''
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512,
    )
    for s in generation_output.sequences:
        print(tokenizer.decode(s))

Now try to create a resume. It looks too simple.

In [29]:
chat("Write an Resume to apply for a Machine Learning Job.")

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    Write an Resume to apply for a Machine Learning Job.

    ### Response:
    
        Name: John Doe 
        
        Email Address: john@doe.com 
         
        Phone Number: (123)456-7890 
          
        Education: Bachelor of Science in Computer Engineering, University of California Berkeley 
            
        Experience: 
            - Intern at Google Inc., San Francisco CA  
             
                Developed machine learning models and algorithms using TensorFlow and Keras libraries 
                
            - Research Assistant at Stanford University, Palo Alto CA 
              
                Conducted research on deep reinforcement learning techniques with applications in autonomous driving 
                
            - Software Engineer at Microsoft Corporation, Redmond WA 
              
                Des

Ask Alpaca to write an Email to Elon Musk to apply for a Machine Learning Job. It looks reasonable except the information is not real.

In [30]:
chat("Write an Email to Elon Musk to apply for a Machine Learning Job.")

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    Write an Email to Elon Musk to apply for a Machine Learning Job.

    ### Response:
    
        Dear Mr. Musk, 
        
I am writing this email in order to express my interest in applying for your open position as a machine learning engineer at SpaceX. I have been working with deep neural networks and reinforcement learning algorithms since graduating from MIT last year. My research has focused on developing novel methods of training recurrent neural network models using adversarial losses. In addition, I have experience building large-scale distributed systems for real time data processing and analysis. 
  
My CV can be found below. Please let me know if you would like any additional information or clarification regarding my qualifications. Thank you very much for considering my application!


Ask alpaca to create an image classifier using Tensorflow.

In [31]:
chat("Create an image classifier using Tensorflow.")

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    Create an image classifier using Tensorflow.

    ### Response:
    
        # import tensorflow as tf 
        
        from tensorflow.keras.models import Sequential, Model
        model = Sequential()
        model.add(Dense(1024))
        model.add(Activation('relu'))
        model.add(Dropout(.5))
        model.add(Flatten())
        model.add(Dense(3))
        model.compile(loss='mse', optimizer=RMSprop(), metrics=[accuracy])



In [ ]:
chat("Create an image classifier using Tensorflow.")